In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:07<00:00, 3381839.85it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 163925.91it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 2076199.40it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20427887.41it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size= batch_size)
for x,y in test_dataloader:
    print(f"shape of X [N C H W ]: {x.shape}")
    print(f"Shape of y : {y.shape} {y.dtype}")
    break

shape of X [N C H W ]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


In [5]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device {device} device ")

using device cuda device 


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model =  NeuralNetwork().to(device)
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size  = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch %100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss : {loss:>7f} [{current : >5d}/{size:>5d}]")
        

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad() :
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error : \n Accuracy : {(100 * correct):>0.1f}% Avg loss : {test_loss:>8f} \n")
    

In [10]:
epochs = 5 
for t in range(epochs):
    print(f"Epoch {t+1} \n --------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done !")


Epoch 1 
 --------------------
loss : 2.297040 [   64/60000]
loss : 2.289139 [ 6464/60000]
loss : 2.275467 [12864/60000]
loss : 2.266961 [19264/60000]
loss : 2.246474 [25664/60000]
loss : 2.228051 [32064/60000]
loss : 2.227045 [38464/60000]
loss : 2.194848 [44864/60000]
loss : 2.185894 [51264/60000]
loss : 2.171519 [57664/60000]
Test error : 
 Accuracy : 59.1% Avg loss : 2.158974 

Epoch 2 
 --------------------
loss : 2.165269 [   64/60000]
loss : 2.157398 [ 6464/60000]
loss : 2.100524 [12864/60000]
loss : 2.113368 [19264/60000]
loss : 2.065407 [25664/60000]
loss : 2.019649 [32064/60000]
loss : 2.039565 [38464/60000]
loss : 1.959673 [44864/60000]
loss : 1.958763 [51264/60000]
loss : 1.907131 [57664/60000]
Test error : 
 Accuracy : 61.8% Avg loss : 1.891229 

Epoch 3 
 --------------------
loss : 1.922854 [   64/60000]
loss : 1.895874 [ 6464/60000]
loss : 1.769016 [12864/60000]
loss : 1.807185 [19264/60000]
loss : 1.712573 [25664/60000]
loss : 1.668448 [32064/60000]
loss : 1.686687 [38

In [11]:
torch.save(model.state_dict(), "model.pth")
print("model saved to model.pth")


model saved to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted : {predicted}, Actual : {actual}")


Predicted : Ankle boot, Actual : Ankle boot
